In [1]:
from scene_splitter.clip_scene_splitter import CLIPSceneSplitter
from tracking.detector import ObjectDetector
from tracking.tracker import DeepSortTracker
from utils.video_tools import save_scenes_ffmpeg
from utils.audio_tools import extract_audio_ffmpeg
from audio.audio_features import get_audio_activity, transcribe_audio_whisper
from sentence_transformers import SentenceTransformer, util
from insightface.app import FaceAnalysis
from sklearn.cluster import DBSCAN
from tqdm import tqdm
import numpy as np
import onnxruntime
import cv2
import re

/home/artniz/hackatons/kion/kionvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(onnxruntime.get_device())

CPU


In [3]:
def analyze_video(video_path, scene_splitter, detector, tracker, every_n_frames=15):
    scenes = scene_splitter.detect_scenes(video_path, every_n_frames=every_n_frames)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    scene_data = []
    track_faces = {}

    for (start, end) in tqdm(scenes, desc="Analyzing scenes"):
        cap.set(cv2.CAP_PROP_POS_MSEC, start * 1000)
        frame_idx = int(start * fps)
        end_idx = int(end * fps)
        track_ids_in_scene = set()

        while frame_idx < end_idx:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx % every_n_frames == 0:
                detections = detector.detect(frame)
                tracks = tracker.update(detections, frame)

                for t in tracks:
                    track_id = t['track_id']
                    x1, y1, x2, y2 = map(int, t['bbox'])

                    # Добавим в сцену
                    track_ids_in_scene.add(track_id)

                    # Сохраняем лицо
                    face_crop = frame[y1:y2, x1:x2]
                    if face_crop.size == 0:
                        continue

                    if track_id not in track_faces:
                        track_faces[track_id] = []

                    track_faces[track_id].append(face_crop)

            frame_idx += 1

        scene_data.append({
            "start": start,
            "end": end,
            "track_ids": list(track_ids_in_scene)
        })

    cap.release()
    return scene_data, track_faces

In [4]:
def get_avg_embeddings(track_faces, face_app):
    track_embeddings = {}

    for track_id, crops in tqdm(track_faces.items(), desc="Analyzing faces"):
        embeddings = []
        for face in crops:
            faces = face_app.get(face)
            if faces:
                embeddings.append(faces[0].embedding)

        if embeddings:
            avg_emb = np.mean(embeddings, axis=0)
            track_embeddings[track_id] = avg_emb

    return track_embeddings

In [5]:
def cluster_track_ids(track_embeddings):
    X = list(track_embeddings.values())
    ids = list(track_embeddings.keys())

    clustering = DBSCAN(eps=0.7, min_samples=1, metric='cosine').fit(X)

    # Преобразование track_id → person_X
    track_id_to_person = {
        track_id: f"person_{label}"
        for track_id, label in zip(ids, clustering.labels_)
    }
    return track_id_to_person

In [6]:
def enrich_scenes_with_characters(scene_data, track_id_to_person):
    for scene in scene_data:
        scene['characters'] = list({
            track_id_to_person.get(tid)
            for tid in scene['track_ids']
            if tid in track_id_to_person
        })
    return scene_data

In [7]:
sentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
def normalize_text(t):
    return re.sub(r'[^\w\s]', '', t.lower()).strip()

In [9]:
def get_text_score(a, b):
    a, b = normalize_text(a), normalize_text(b)
    if not a or not b:
        return 0.0
    a_embed = sentenceTransformer.encode(a, convert_to_tensor=True)
    b_embed = sentenceTransformer.encode(b, convert_to_tensor=True)
    return float(util.cos_sim(a_embed, b_embed)[0][0])

In [10]:
def jaccard_similarity(a, b):
    set_a, set_b = set(a), set(b)
    intersection = set_a & set_b
    union = set_a | set_b
    return len(intersection) / len(union) if union else 0

In [11]:
def group_semantic_scenes(scene_data, char_thresh=0.5, text_thresh=0.55, audio_thresh=0.02):
    grouped = []
    buffer = [scene_data[0]]

    def get_identities(scene):
        return scene.get('characters', []) or scene.get('track_ids', [])

    prev_text = scene_data[0]['transcript']
    prev_embed = sentenceTransformer.encode(normalize_text(prev_text), convert_to_tensor=True)

    for curr in scene_data[1:]:
        last = buffer[-1]
        curr_ids = get_identities(curr)
        last_ids = get_identities(last)
        char_score = jaccard_similarity(curr_ids, last_ids)

        # Нормализованный текст и эмбеддинг
        curr_text = curr['transcript']
        text_score = get_text_score(prev_text, curr_text)

        audio_diff = abs(curr['avg_rms'] - last['avg_rms'])

        if char_score >= char_thresh and text_score >= text_thresh and audio_diff <= audio_thresh:
            buffer.append(curr)
            prev_text = " ".join([prev_text, curr_text])  # копим контекст
        else:
            grouped.append({
                'start': buffer[0]['start'],
                'end': buffer[-1]['end'],
                'characters': sorted(set().union(*(get_identities(b) for b in buffer))),
                'transcript': " ".join(b['transcript'] for b in buffer),
                'avg_rms': float(np.mean([b['avg_rms'] for b in buffer]))
            })
            buffer = [curr]
            prev_text = curr['transcript']

    # Последний кусок
    if buffer:
        grouped.append({
            'start': buffer[0]['start'],
            'end': buffer[-1]['end'],
            'characters': sorted(set().union(*(get_identities(b) for b in buffer))),
            'transcript': " ".join(b['transcript'] for b in buffer),
            'avg_rms': float(np.mean([b['avg_rms'] for b in buffer]))
        })

    return grouped

In [12]:
splitter = CLIPSceneSplitter()
detector = ObjectDetector()
tracker = DeepSortTracker()

In [13]:
video_path = "test.mp4"

In [14]:
scenes, track_faces = analyze_video(video_path, splitter, detector, tracker)

Analyzing scenes: 100%|██████████| 43/43 [00:14<00:00,  2.95it/s]


In [15]:
face_app = FaceAnalysis(name="buffalo_l", providers=["CPUExecutionProvider"])
face_app.prepare(ctx_id=-1)  # 0 — для GPU, или -1 для CPU

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/artniz/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/artniz/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/artniz/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/artniz/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/artniz/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

In [16]:
track_embeddings = get_avg_embeddings(track_faces, face_app)

Analyzing faces: 100%|██████████| 19/19 [02:43<00:00,  8.62s/it]


In [17]:
track_id_to_person = cluster_track_ids(track_embeddings)

In [18]:
scene_data = enrich_scenes_with_characters(scenes, track_id_to_person)

In [19]:
# for (start, end) in scenes:
#     print(f"start {start / 60}, end {end / 60}")
scene_data

[{'start': 0.0,
  'end': 15.995287617650654,
  'track_ids': ['19', '2', '1', '11', '12', '4', '26', '29'],
  'characters': ['person_0', 'person_1']},
 {'start': 15.995287617650654,
  'end': 24.492784164527563,
  'track_ids': ['19', '2', '45', '1', '11', '12', '50', '4', '26', '29'],
  'characters': ['person_0', 'person_1']},
 {'start': 24.492784164527563,
  'end': 32.49042797335289,
  'track_ids': ['45', '1', '50', '26', '29', '57'],
  'characters': ['person_0', 'person_1']},
 {'start': 32.49042797335289,
  'end': 40.48807178217822,
  'track_ids': ['45', '1', '50', '26', '57'],
  'characters': ['person_0', 'person_1']},
 {'start': 40.48807178217822,
  'end': 53.98409570957096,
  'track_ids': ['1', '45', '57'],
  'characters': ['person_0', 'person_1']},
 {'start': 53.98409570957096,
  'end': 54.48394844762254,
  'track_ids': ['1'],
  'characters': ['person_0']},
 {'start': 54.48394844762254,
  'end': 54.98380118567412,
  'track_ids': ['1'],
  'characters': ['person_0']},
 {'start': 54.9

In [20]:
def print_scenes_formatted(scenes):
    for i, scene in enumerate(scenes):
        print(f"\n🎬 Сцена {i + 1}")
        print(f"⏱ Время: {scene['start']:.2f} – {scene['end']:.2f} сек")
        print(f"🧍 Персонажи (characters): {', '.join(scene['characters'])}")
        print(f"🔊 Средняя громкость: {scene['avg_rms']:.4f}")
        print(f"🗣 Реплика:\n\"{scene['transcript'].strip()}\"")
        print("-" * 60)

In [21]:
audio_path = extract_audio_ffmpeg(video_path)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [22]:
whisper_result = transcribe_audio_whisper(audio_path, model_size="tiny")
segments = whisper_result['segments']

In [23]:
segments

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 7.5,
  'text': ' Рванули в России за славой, вормию нас, кто угодно, немцы и таляны даже шавой цары, 600 тысяч тыков.',
  'tokens': [50364,
   6325,
   859,
   1416,
   28646,
   740,
   29007,
   4396,
   4766,
   1828,
   1700,
   11,
   740,
   1717,
   6293,
   1148,
   6519,
   11,
   12278,
   20392,
   44356,
   11,
   13166,
   14180,
   1006,
   1069,
   1218,
   681,
   1834,
   11210,
   5941,
   1828,
   1700,
   5188,
   48632,
   11,
   11849,
   25025,
   5991,
   7718,
   13,
   50739],
  'temperature': 0.0,
  'avg_logprob': -0.5242122736844149,
  'compression_ratio': 1.6371681415929205,
  'no_speech_prob': 0.06060207635164261},
 {'id': 1,
  'seek': 0,
  'start': 16.0,
  'end': 24.0,
  'text': ' Истет, что за дерьмо, Миссия на полион, ты нам дороги на карте рисовали, дорог не хиранет, наибалполучается.',
  'tokens': [51164,
   3272,
   982,
   1094,
   11,
   2143,
   4396,
   1070,
   9352,
   20455,
   11,
   3493,
   3

In [24]:
energy = get_audio_activity(audio_path, frame_duration=1.0)

In [25]:
energy

[np.float32(0.057765886),
 np.float32(0.04959505),
 np.float32(0.060569875),
 np.float32(0.058170218),
 np.float32(0.054325342),
 np.float32(0.050869744),
 np.float32(0.050934855),
 np.float32(0.05627119),
 np.float32(0.061195992),
 np.float32(0.04948571),
 np.float32(0.05220315),
 np.float32(0.05306321),
 np.float32(0.028229339),
 np.float32(0.05438753),
 np.float32(0.06009945),
 np.float32(0.046787042),
 np.float32(0.04923539),
 np.float32(0.058251817),
 np.float32(0.0592843),
 np.float32(0.061375998),
 np.float32(0.056080762),
 np.float32(0.059466198),
 np.float32(0.054558393),
 np.float32(0.036828287),
 np.float32(0.05987126),
 np.float32(0.0570157),
 np.float32(0.043265566),
 np.float32(0.061023816),
 np.float32(0.0630757),
 np.float32(0.019734358),
 np.float32(0.07957022),
 np.float32(0.025993071),
 np.float32(0.039030764),
 np.float32(0.05032348),
 np.float32(0.05132471),
 np.float32(0.06694859),
 np.float32(0.05106287),
 np.float32(0.07096474),
 np.float32(0.060370494),
 np.flo

In [47]:
scene_data_copy = scene_data.copy()

In [48]:
# TODO
def enrich_scenes_with_audio(scenes, segments, energy, frame_duration=1.0):
    def clip_text_to_scene(seg, start, end):
        seg_start, seg_end = seg["start"], seg["end"]
        overlap_start = max(start, seg_start)
        overlap_end = min(end, seg_end)
        overlap_dur = max(0.0, overlap_end - overlap_start)
        total_dur = seg_end - seg_start

        if overlap_dur == 0 or total_dur == 0:
            return ""

        words = seg["text"].strip().split()
        if len(words) <= 2:
            return seg["text"].strip()

        ratio = overlap_dur / total_dur
        count = max(1, int(len(words) * ratio))
        return " ".join(words[:count]) if overlap_start == seg_start else " ".join(words[-count:])

    for scene in scenes:
        start, end = scene["start"], scene["end"]

        # Текст с частичным включением
        scene_texts = [
            clip_text_to_scene(seg, start, end)
            for seg in segments
            if not (seg["end"] < start or seg["start"] > end)
        ]
        scene["transcript"] = " ".join(filter(None, scene_texts)).strip()

        # Энергия
        start_idx = max(0, int(start // frame_duration))
        end_idx = min(len(energy) - 1, int(end // frame_duration))
        scene_energy = energy[start_idx:end_idx+1] if end_idx >= start_idx else []
        scene["avg_rms"] = float(np.mean(scene_energy)) if scene_energy else 0.0

    return scenes

In [49]:
scenes_enrich = enrich_scenes_with_audio(scene_data_copy, segments, energy)

In [50]:
scenes_enrich

[{'start': 0.0,
  'end': 15.995287617650654,
  'track_ids': ['19', '2', '1', '11', '12', '4', '26', '29'],
  'characters': ['person_0', 'person_1'],
  'transcript': 'Рванули в России за славой, вормию нас, кто угодно, немцы и таляны даже шавой цары, 600 тысяч тыков.',
  'avg_rms': 0.052747100591659546},
 {'start': 15.995287617650654,
  'end': 24.492784164527563,
  'track_ids': ['19', '2', '45', '1', '11', '12', '50', '4', '26', '29'],
  'characters': ['person_0', 'person_1'],
  'transcript': 'Истет, что за дерьмо, Миссия на полион, ты нам дороги на карте рисовали, дорог не хиранет, наибалполучается. До',
  'avg_rms': 0.054173946380615234},
 {'start': 24.492784164527563,
  'end': 32.49042797335289,
  'track_ids': ['45', '1', '50', '26', '29', '57'],
  'characters': ['person_0', 'person_1'],
  'transcript': 'смоленской остелась, немнёжка. Бонья сказяли, что это приключение на 20 минут. Скоро домой с победой. Вы',
  'avg_rms': 0.04984227567911148},
 {'start': 32.49042797335289,
  'end': 4

In [51]:
scenes_enrich_copy = scenes_enrich.copy()

In [52]:
scenes_enrich_copy

[{'start': 0.0,
  'end': 15.995287617650654,
  'track_ids': ['19', '2', '1', '11', '12', '4', '26', '29'],
  'characters': ['person_0', 'person_1'],
  'transcript': 'Рванули в России за славой, вормию нас, кто угодно, немцы и таляны даже шавой цары, 600 тысяч тыков.',
  'avg_rms': 0.052747100591659546},
 {'start': 15.995287617650654,
  'end': 24.492784164527563,
  'track_ids': ['19', '2', '45', '1', '11', '12', '50', '4', '26', '29'],
  'characters': ['person_0', 'person_1'],
  'transcript': 'Истет, что за дерьмо, Миссия на полион, ты нам дороги на карте рисовали, дорог не хиранет, наибалполучается. До',
  'avg_rms': 0.054173946380615234},
 {'start': 24.492784164527563,
  'end': 32.49042797335289,
  'track_ids': ['45', '1', '50', '26', '29', '57'],
  'characters': ['person_0', 'person_1'],
  'transcript': 'смоленской остелась, немнёжка. Бонья сказяли, что это приключение на 20 минут. Скоро домой с победой. Вы',
  'avg_rms': 0.04984227567911148},
 {'start': 32.49042797335289,
  'end': 4

In [53]:
scenes_grouped = group_semantic_scenes(scenes_enrich_copy)

In [56]:
def clean_and_merge_short_scenes(scenes, min_duration=2.0, min_words=3):
    cleaned = []
    buffer = None

    def merge_scene(a, b):
        return {
            "start": a["start"],
            "end": b["end"],
            "characters": sorted(set(a.get("characters", []) + b.get("characters", []))),
            "transcript": " ".join([a["transcript"], b["transcript"]]).strip(),
            "avg_rms": float(np.mean([a["avg_rms"], b["avg_rms"]]))
        }

    for scene in scenes:
        duration = scene["end"] - scene["start"]
        word_count = len(scene["transcript"].strip().split())

        # Пропуск пустых или бессмысленных сцен
        if word_count < min_words or duration < 0.5:
            if cleaned:
                cleaned[-1] = merge_scene(cleaned[-1], scene)
            continue

        if duration < min_duration:
            if cleaned:
                cleaned[-1] = merge_scene(cleaned[-1], scene)
            else:
                buffer = scene
        else:
            cleaned.append(scene)

    # Добавляем буфер, если остался
    if buffer:
        if cleaned:
            cleaned[-1] = merge_scene(cleaned[-1], buffer)
        else:
            cleaned.append(buffer)

    return cleaned

In [57]:
scenes_grouped = clean_and_merge_short_scenes(scenes_grouped, min_duration=2.0, min_words=3)

In [58]:
scenes_grouped

[{'start': 0.0,
  'end': 32.49042797335289,
  'characters': ['person_0', 'person_1'],
  'transcript': 'Рванули в России за славой, вормию нас, кто угодно, немцы и таляны даже шавой цары, 600 тысяч тыков. Истет, что за дерьмо, Миссия на полион, ты нам дороги на карте рисовали, дорог не хиранет, наибалполучается. До смоленской остелась, немнёжка. Бонья сказяли, что это приключение на 20 минут. Скоро домой с победой. Вы',
  'avg_rms': 0.05225444088379542},
 {'start': 32.49042797335289,
  'end': 55.983506661777284,
  'characters': ['person_0', 'person_1'],
  'transcript': 'как гляньте, что это творите? Моленск но это пистет. Рвуски нас чуть не вайпнули. Еще император чуть не поджарился в шатре. Это просто вилы, Рвуски исполели все свои избючки. Все вебещали нам богатые Рвуски деревня, но мы жирем землю пеплом. пеплом. пеплом. Вот Сия.',
  'avg_rms': 0.05348429720227917},
 {'start': 55.983506661777284,
  'end': 63.48129773255103,
  'characters': ['person_0'],
  'transcript': 'обещанные запа

In [59]:
print_scenes_formatted(scenes_grouped)


🎬 Сцена 1
⏱ Время: 0.00 – 32.49 сек
🧍 Персонажи (characters): person_0, person_1
🔊 Средняя громкость: 0.0523
🗣 Реплика:
"Рванули в России за славой, вормию нас, кто угодно, немцы и таляны даже шавой цары, 600 тысяч тыков. Истет, что за дерьмо, Миссия на полион, ты нам дороги на карте рисовали, дорог не хиранет, наибалполучается. До смоленской остелась, немнёжка. Бонья сказяли, что это приключение на 20 минут. Скоро домой с победой. Вы"
------------------------------------------------------------

🎬 Сцена 2
⏱ Время: 32.49 – 55.98 сек
🧍 Персонажи (characters): person_0, person_1
🔊 Средняя громкость: 0.0535
🗣 Реплика:
"как гляньте, что это творите? Моленск но это пистет. Рвуски нас чуть не вайпнули. Еще император чуть не поджарился в шатре. Это просто вилы, Рвуски исполели все свои избючки. Все вебещали нам богатые Рвуски деревня, но мы жирем землю пеплом. пеплом. пеплом. Вот Сия."
------------------------------------------------------------

🎬 Сцена 3
⏱ Время: 55.98 – 63.48 сек
🧍 Персон

In [60]:
output_dir = "output_scenes"

save_scenes_ffmpeg(video_path, scenes_grouped, output_dir)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab